In [1]:
# Import packages
import pandas as pd
import numpy as np
import xgboost as xgb
from lightgbm import LGBMRegressor
import optuna
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# To Do
# -Load in data
# -Create training/cv/test split. Test should likely be in chronological order to avoid any data leakage
# -List of variables for model, both pitch char and non pitch char models
# -Fit initial XGBoost model and take a peek at results to see if things are looking okay
# -Implememt optuna

In [3]:
full_pitch_df = pd.read_csv("final_pitch_data_new.csv").drop(columns = ["Unnamed: 0"])

In [4]:
full_pitch_df.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,events_in_play,batted_ball_type,fast_pitch_avg_velo,delta_diff,rv,tto,batter_right,p_throws_right,description3,rv2
0,FF,2022-04-26,90.6,-1.94,6.53,"McKenzie, Triston",621493,663474,NaN,called_strike,...,out,NaN,92.251064,0.002177,-0.083518,1,1,1,strike,-0.083516
1,FF,2022-04-26,92.6,-1.90,6.39,"McKenzie, Triston",621493,663474,NaN,called_strike,...,out,NaN,92.251064,0.018665,-0.083518,1,1,1,strike,-0.083516
2,FF,2022-04-26,93.0,-2.04,6.42,"McKenzie, Triston",621493,663474,NaN,foul,...,out,fb,92.251064,0.000000,0.000000,1,1,1,fb,0.025442
3,FF,2022-04-26,92.2,-1.97,6.43,"McKenzie, Triston",621493,663474,NaN,foul,...,out,pu,92.251064,0.000000,0.000000,1,1,1,pu,-0.100858
4,CU,2022-04-26,78.8,-2.14,6.48,"McKenzie, Triston",621493,663474,field_out,hit_into_play,...,out,fb,92.251064,0.020345,-0.226302,1,1,1,fb,0.025442


In [5]:
pitch_char_list = ["release_speed", "release_pos_x", "release_pos_z", "release_extension", "pfx_x", "pfx_z", 
                   "release_spin_rate", "fast_pitch_avg_velo"]
location_list = ["plate_x", "plate_z"]
context_list = ["balls", "strikes", "batter_right", "p_throws_right", "sz_top", "sz_bot", "tto"]
# context_list = ["balls", "strikes", "outs_when_up", "batter_right", "p_throws_right", "runner1b", "runner2b", "runner3b", 
#                 "sz_top", "sz_bot", "tto"]

In [50]:
# # Turn categorical variables (stand and p_throws) into binary variables
# full_pitch_df["batter_right"] = (full_pitch_df["stand"] == "R").replace({True: 1, False: 0})
# full_pitch_df["p_throws_right"] = (full_pitch_df["p_throws"] == "R").replace({True: 1, False: 0})
# #full_pitch_df[context_list]

In [51]:
# # One last data filter. No 3 strike or 4 ball pitches
# full_pitch_df = full_pitch_df[(full_pitch_df["balls"] != 4) & (full_pitch_df["strikes"] != 3)]

In [6]:
# Sort for train/cv/test split
full_pitch_df = full_pitch_df.sort_values(["game_date", "game_pk", "pitcher", "at_bat_number", "pitch_number"])

In [7]:
# Split by pitch grouping

# Fastball pitches
fb_pitch_type = ["SI", "FC", "FF"]

# Offspeed pitches
ofs_pitch_type = ["FS", "CH", "EP", "FA", "KN"]

# Breaking ball pitches
bb_pitch_type = ["SL", "ST", "CU", "KC", "CS"]

fb_df = full_pitch_df[full_pitch_df["pitch_type"].isin(fb_pitch_type) & (full_pitch_df["p_throws_right"] == 1) & (full_pitch_df["batter_right"] == 1)]
ofs_df = full_pitch_df[full_pitch_df["pitch_type"].isin(ofs_pitch_type)]
bb_df = full_pitch_df[full_pitch_df["pitch_type"].isin(bb_pitch_type)]


In [54]:
# Start with fastball model
fb_train_df = fb_df.iloc[0:round(len(fb_df) * 0.64), :]
fb_cv_df = fb_df.iloc[(round(len(fb_df) * 0.64)):round(len(fb_df) * 0.8), :]
fb_test_df = fb_df.iloc[(round(len(fb_df) * 0.8)):, :]
(len(fb_train_df) + len(fb_cv_df) + len(fb_test_df)) == len(fb_df)

True

In [96]:
# Establish hyperparameter range
def fb_objective(trial):
    
    xgb_fit = xgb.XGBRegressor(n_estimators = int(trial.suggest_float("n_estimators", 10, 500, step = 1)), 
                               max_depth = int(trial.suggest_float("max_depth", 3, 10, step = 1)),
                               learning_rate = trial.suggest_float("learning_rate", 0.005, 0.3), 
                               subsample = trial.suggest_float("subsample", 0.6, 1),
                               tree_method = "hist",
                               alpha = trial.suggest_float("alpha", 0, 1))
    
    xgb_fit.fit(fb_train_df[pitch_char_list + context_list], fb_train_df["rv2"])
    
    pred = xgb_fit.predict(fb_cv_df[pitch_char_list + context_list])
    fb_cv_df.loc[:,"pred"] = pred
    
    pred_full = xgb_fit.predict(fb_train_df[pitch_char_list + context_list])
    fb_train_df.loc[:,"pred"] = pred_full
    return((np.mean((pred - fb_cv_df["rv2"])**2))**0.5)
    

In [23]:
(np.mean((fb_cv_df["rv2"].mean() - fb_cv_df["rv2"])**2))**0.5


NameError: name 'fb_cv_df' is not defined

In [98]:
fb_train_df["rv2"].isna().sum()

0

In [99]:
import time
start_time = time.time()
fb_study = optuna.create_study()
 # The optimization finishes after evaluating 1000 times or 3 seconds.
fb_study.optimize(fb_objective, n_trials = 30)
print(time.time() - start_time)


[I 2024-01-05 16:01:00,679] A new study created in memory with name: no-name-89f21ecf-a12b-428b-a981-20e13a7cadfb
C:\Users\ishaa\AppData\Local\Temp\ipykernel_83508\2416371174.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fb_cv_df.loc[:,"pred"] = pred
C:\Users\ishaa\AppData\Local\Temp\ipykernel_83508\2416371174.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fb_train_df.loc[:,"pred"] = pred_full
[I 2024-01-05 16:01:26,104] Trial 0 finished with value: 0.08902866849607166 and parameters: {'n_estim

C:\Users\ishaa\AppData\Local\Temp\ipykernel_83508\2416371174.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fb_cv_df.loc[:,"pred"] = pred
C:\Users\ishaa\AppData\Local\Temp\ipykernel_83508\2416371174.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fb_train_df.loc[:,"pred"] = pred_full
[I 2024-01-05 16:02:51,764] Trial 8 finished with value: 0.08263329759757851 and parameters: {'n_estimators': 189.0, 'max_depth': 5.0, 'learning_rate': 0.15447230800701325, 'subsample': 0.9393168478845771, 'alpha': 0

C:\Users\ishaa\AppData\Local\Temp\ipykernel_83508\2416371174.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fb_cv_df.loc[:,"pred"] = pred
C:\Users\ishaa\AppData\Local\Temp\ipykernel_83508\2416371174.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fb_train_df.loc[:,"pred"] = pred_full
[I 2024-01-05 16:03:22,392] Trial 16 finished with value: 0.0824711613699786 and parameters: {'n_estimators': 320.0, 'max_depth': 4.0, 'learning_rate': 0.11994979546285442, 'subsample': 0.8348826263977677, 'alpha': 0

C:\Users\ishaa\AppData\Local\Temp\ipykernel_83508\2416371174.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fb_cv_df.loc[:,"pred"] = pred
C:\Users\ishaa\AppData\Local\Temp\ipykernel_83508\2416371174.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fb_train_df.loc[:,"pred"] = pred_full
[I 2024-01-05 16:03:49,955] Trial 24 finished with value: 0.08230224304210933 and parameters: {'n_estimators': 75.0, 'max_depth': 4.0, 'learning_rate': 0.09967845657332085, 'subsample': 0.816533382361344, 'alpha': 0.

194.83599495887756


In [100]:
test = fb_train_df.groupby(["player_name", "pitch_type"]).aggregate({"rv":"mean", "rv2":"mean", "pred": "mean", "description": "count"})
test2 = test[test["description"] > 150]
print(test2["rv2"].corr(test2["pred"]))
test2.sort_values("pred").sort_values("pred").head(50)

0.7438418048540524


,,rv,rv2,pred,description
player_name,pitch_type,,,,
"Webb, Logan",SI,-0.006156,-0.014140,-0.014932,371
"Jansen, Kenley",FC,-0.013824,-0.020057,-0.014523,224
"Thompson, Ryan",SI,-0.013138,-0.013645,-0.013023,202
"Kluber, Corey",FC,-0.014580,-0.019709,-0.012274,272
"Ramirez, Noé",FF,-0.002728,-0.019444,-0.012013,164
"Darvish, Yu",FC,0.010073,-0.013361,-0.011972,289
"Cimber, Adam",SI,-0.000117,-0.012571,-0.011611,210
"Holmes, Clay",SI,-0.018292,-0.011326,-0.011186,327
"Dunning, Dane",SI,-0.005697,-0.015427,-0.010873,367


In [ ]:
(np.mean((fb_cv_df["rv2"].mean() - fb_cv_df["rv"])**2))**0.5

In [8]:
# Offspeed model
ofs_train_df = ofs_df.iloc[0:round(len(ofs_df) * 0.64), :]
ofs_cv_df = ofs_df.iloc[(round(len(ofs_df) * 0.64)):round(len(ofs_df) * 0.8), :]
ofs_test_df = ofs_df.iloc[(round(len(ofs_df) * 0.8)):, :]
(len(ofs_train_df) + len(ofs_cv_df) + len(ofs_test_df)) == len(ofs_df)

True

In [22]:
(np.mean((ofs_cv_df["rv2"].mean() - ofs_cv_df["rv2"])**2))**0.5

0.07717416505356085

In [10]:
# Establish hyperparameter range
def ofs_objective(trial):
    
    xgb_fit = xgb.XGBRegressor(n_estimators = int(trial.suggest_float("n_estimators", 10, 500, step = 1)), 
                               max_depth = int(trial.suggest_float("max_depth", 3, 10, step = 1)),
                               learning_rate = trial.suggest_float("learning_rate", 0.005, 0.3), 
                               subsample = trial.suggest_float("subsample", 0.6, 1),
                               tree_method = "hist",
                               alpha = trial.suggest_float("alpha", 0, 1))
    
    xgb_fit.fit(ofs_train_df[pitch_char_list + context_list], ofs_train_df["rv2"])
    
    pred = xgb_fit.predict(ofs_cv_df[pitch_char_list + context_list])
    ofs_cv_df.loc[:,"pred"] = pred
    
    pred_full = xgb_fit.predict(ofs_train_df[pitch_char_list + context_list])
    ofs_train_df.loc[:,"pred"] = pred_full
    return((np.mean((pred - ofs_cv_df["rv2"])**2))**0.5)
    

In [12]:
import time as time
start_time = time.time()
ofs_study = optuna.create_study()
 # The optimization finishes after evaluating 1000 times or 3 seconds.
ofs_study.optimize(ofs_objective, n_trials = 30)
print(time.time() - start_time)


[I 2024-01-16 21:40:08,452] A new study created in memory with name: no-name-9aaa4275-b463-4ff3-8e94-1642a35dbcfc
C:\Users\ishaa\AppData\Local\Temp\ipykernel_21924\3487623420.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ofs_cv_df.loc[:,"pred"] = pred
C:\Users\ishaa\AppData\Local\Temp\ipykernel_21924\3487623420.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ofs_train_df.loc[:,"pred"] = pred_full
[I 2024-01-16 21:40:09,613] Trial 0 finished with value: 0.25865023494114253 and parameters: {'n_est

C:\Users\ishaa\AppData\Local\Temp\ipykernel_21924\3487623420.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ofs_cv_df.loc[:,"pred"] = pred
C:\Users\ishaa\AppData\Local\Temp\ipykernel_21924\3487623420.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ofs_train_df.loc[:,"pred"] = pred_full
[I 2024-01-16 21:40:35,083] Trial 8 finished with value: 0.08026759678453754 and parameters: {'n_estimators': 134.0, 'max_depth': 9.0, 'learning_rate': 0.23366710248843953, 'subsample': 0.9112364385840834, 'alpha':

[I 2024-01-16 21:40:56,622] Trial 15 finished with value: 0.07645185815742628 and parameters: {'n_estimators': 244.0, 'max_depth': 6.0, 'learning_rate': 0.04862957134997099, 'subsample': 0.8388145291551864, 'alpha': 0.6638747065003979}. Best is trial 6 with value: 0.07637754812481463.
C:\Users\ishaa\AppData\Local\Temp\ipykernel_21924\3487623420.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ofs_cv_df.loc[:,"pred"] = pred
C:\Users\ishaa\AppData\Local\Temp\ipykernel_21924\3487623420.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

C:\Users\ishaa\AppData\Local\Temp\ipykernel_21924\3487623420.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ofs_cv_df.loc[:,"pred"] = pred
C:\Users\ishaa\AppData\Local\Temp\ipykernel_21924\3487623420.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ofs_train_df.loc[:,"pred"] = pred_full
[I 2024-01-16 21:41:32,175] Trial 24 finished with value: 0.07639073085876233 and parameters: {'n_estimators': 207.0, 'max_depth': 4.0, 'learning_rate': 0.07209363211210379, 'subsample': 0.8893953361268893, 'alpha'

96.39708614349365


In [17]:
test = ofs_train_df.groupby(["player_name", "pitch_type"]).aggregate({"rv":"mean", "rv2":"mean", "pred": "mean", "description": "count"})
test2 = test[test["description"] > 200]
print(test2["rv2"].corr(test2["pred"]))
test2.sort_values("pred").sort_values("pred").head(50)

0.745232857094827


,,rv,rv2,pred,description
player_name,pitch_type,,,,
"Williams, Devin",CH,-0.008964,-0.013898,-0.011380,375
"Alcantara, Sandy",CH,-0.022798,-0.013672,-0.007554,581
"Ashby, Aaron",CH,0.006795,-0.009346,-0.007113,276
"McClanahan, Shane",CH,-0.021141,-0.013789,-0.007014,422
"Wells, Tyler",CH,-0.009835,-0.014894,-0.005483,268
"Beeks, Jalen",CH,-0.000068,-0.008890,-0.005199,315
"Richards, Trevor",CH,-0.007718,-0.011019,-0.004935,267
"Eovaldi, Nathan",FS,-0.016527,-0.014578,-0.003882,261
"Matz, Steven",CH,-0.009100,-0.003021,-0.003139,214


In [147]:
# Let's do some exploration!
full_pitch_df.groupby("pitch_type").aggregate({"rv":"mean", "pitch_number":"count", "delta_run_exp": "mean", "pfx_x": "mean",
                                               "pfx_z": "mean", "release_speed": "mean"}).sort_values("rv")



,rv,pitch_number,delta_run_exp,pfx_x,pfx_z,release_speed
pitch_type,,,,,,
SL,-0.000034,145876,-0.002264,3.669776,1.872354,84.610533
ST,0.000205,1273,0.000441,12.833119,-0.880534,81.193480
FS,0.001196,10322,-0.003693,-10.868115,3.904022,87.199341
CU,0.001426,58696,-0.000111,3.787476,-8.977667,79.058815
SI,0.001624,111008,0.000382,-6.374230,8.708303,93.351740
KC,0.002408,13945,-0.000128,5.692027,-10.232941,81.239104
FC,0.003050,50535,0.000133,1.705080,8.036616,88.998148
FF,0.003238,231994,0.000522,-3.127531,16.117360,93.919448
CH,0.005010,79995,0.002219,-4.223877,6.199781,85.330018


In [18]:
# Offspeed model
bb_train_df = bb_df.iloc[0:round(len(bb_df) * 0.64), :]
bb_cv_df = bb_df.iloc[(round(len(bb_df) * 0.64)):round(len(bb_df) * 0.8), :]
bb_test_df = bb_df.iloc[(round(len(bb_df) * 0.8)):, :]
(len(bb_train_df) + len(bb_cv_df) + len(bb_test_df)) == len(bb_df)

True

In [21]:
(np.mean((bb_cv_df["rv2"].mean() - bb_cv_df["rv2"])**2))**0.5

0.07909111239693707

In [26]:
# Establish hyperparameter range
def bb_objective(trial):
    
    xgb_fit = xgb.XGBRegressor(n_estimators = int(trial.suggest_float("n_estimators", 10, 500, step = 1)), 
                               max_depth = int(trial.suggest_float("max_depth", 3, 10, step = 1)),
                               learning_rate = trial.suggest_float("learning_rate", 0.005, 0.3), 
                               subsample = trial.suggest_float("subsample", 0.6, 1),
                               tree_method = "hist",
                               alpha = trial.suggest_float("alpha", 0, 1))
    
    xgb_fit.fit(bb_train_df[pitch_char_list + context_list], bb_train_df["rv2"])
    
    pred = xgb_fit.predict(bb_cv_df[pitch_char_list + context_list])
    bb_cv_df.loc[:,"pred"] = pred
    
    pred_full = xgb_fit.predict(bb_train_df[pitch_char_list + context_list])
    bb_train_df.loc[:,"pred"] = pred_full
    return((np.mean((pred - bb_cv_df["rv2"])**2))**0.5)
    

In [27]:
import time as time
start_time = time.time()
bb_study = optuna.create_study()
 # The optimization finishes after evaluating 1000 times or 3 seconds.
bb_study.optimize(bb_objective, n_trials = 30)
print(time.time() - start_time)


[I 2024-01-17 18:56:44,818] A new study created in memory with name: no-name-2220fbe6-5736-4659-aeb0-d62e964280c9
C:\Users\ishaa\AppData\Local\Temp\ipykernel_21924\2395281121.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bb_cv_df.loc[:,"pred"] = pred
C:\Users\ishaa\AppData\Local\Temp\ipykernel_21924\2395281121.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bb_train_df.loc[:,"pred"] = pred_full
[I 2024-01-17 18:56:46,485] Trial 0 finished with value: 0.07834521010025525 and parameters: {'n_estim

C:\Users\ishaa\AppData\Local\Temp\ipykernel_21924\2395281121.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bb_cv_df.loc[:,"pred"] = pred
C:\Users\ishaa\AppData\Local\Temp\ipykernel_21924\2395281121.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bb_train_df.loc[:,"pred"] = pred_full
[I 2024-01-17 18:57:54,353] Trial 8 finished with value: 0.07824888916686733 and parameters: {'n_estimators': 452.0, 'max_depth': 3.0, 'learning_rate': 0.12554262984645498, 'subsample': 0.851518446878121, 'alpha': 0.

C:\Users\ishaa\AppData\Local\Temp\ipykernel_21924\2395281121.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bb_cv_df.loc[:,"pred"] = pred
C:\Users\ishaa\AppData\Local\Temp\ipykernel_21924\2395281121.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bb_train_df.loc[:,"pred"] = pred_full
[I 2024-01-17 18:59:00,631] Trial 16 finished with value: 0.07898642144440199 and parameters: {'n_estimators': 493.0, 'max_depth': 5.0, 'learning_rate': 0.18513557561306437, 'subsample': 0.8355631185786669, 'alpha': 

C:\Users\ishaa\AppData\Local\Temp\ipykernel_21924\2395281121.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bb_cv_df.loc[:,"pred"] = pred
C:\Users\ishaa\AppData\Local\Temp\ipykernel_21924\2395281121.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bb_train_df.loc[:,"pred"] = pred_full
[I 2024-01-17 18:59:59,171] Trial 24 finished with value: 0.07826121935903899 and parameters: {'n_estimators': 460.0, 'max_depth': 3.0, 'learning_rate': 0.03376556632365339, 'subsample': 0.6354404588330363, 'alpha': 

239.13763999938965


In [34]:
test = bb_cv_df.groupby(["player_name", "pitch_type"]).aggregate({"rv":"mean", "rv2":"mean", "pred": "mean", "description": "count"})
test2 = test[test["description"] > 100]
print(test2["rv2"].corr(test2["pred"]))
test2.sort_values("pred").sort_values("pred").head(50)

0.3134179360003523


,,rv,rv2,pred,description
player_name,pitch_type,,,,
"Ohtani, Shohei",SL,-0.012206,-0.008023,-0.015213,193
"Muñoz, Andrés",SL,-0.040153,-0.028230,-0.014860,107
"Barría, Jaime",SL,0.014674,-0.000948,-0.009969,131
"Burnes, Corbin",CU,0.004758,-0.002244,-0.007980,118
"Cease, Dylan",SL,-0.011244,-0.012899,-0.007231,208
"Sandoval, Patrick",SL,-0.014140,-0.011721,-0.006716,134
"Lange, Alex",CU,0.001188,-0.019349,-0.006191,104
"Cole, Gerrit",SL,-0.024749,-0.011853,-0.005621,107
"Rodón, Carlos",SL,0.006178,-0.000606,-0.005299,162


In [134]:
# Fastball pitches
fb_pitch_type = ["SI", "FC", "FF"]

# Offspeed pitches
ofs_pitch_type = ["FS", "CH", "EP", "FA", "KN"]

# Breaking ball pitches
bb_pitch_type = ["SL", "ST", "CU", "KC", "CS"]

full_pitch_df["rv"].corr(full_pitch_bdf["delta_run_exp2"])

0.8122587256811388

In [146]:
test = full_pitch_df.groupby(["player_name", "pitch_type"]).aggregate({"rv":"mean", "pfx_x":"count"}).reset_index().sort_values("rv")
test[(test["pfx_x"] > 500) & (test["pitch_type"] == "SL")].head(50)

,player_name,pitch_type,rv,pfx_x
830,"Díaz, Edwin O",SL,-0.027370,534
2139,"Muñoz, Andrés",SL,-0.026900,661
2234,"Ohtani, Shohei",SL,-0.022211,984
700,"Darvish, Yu",SL,-0.020545,566
2669,"Scherzer, Max",SL,-0.019808,507
517,"Cease, Dylan",SL,-0.018303,1338
2123,"Murfee, Penn",SL,-0.015831,511
1459,"Jax, Griffin",SL,-0.015327,560
2636,"Sandoval, Patrick",SL,-0.013621,711
191,"Barlow, Scott",SL,-0.013607,534


In [146]:
test = full_pitch_df.groupby(["player_name", "pitch_type"]).aggregate({"rv":"mean", "pfx_x":"count"}).reset_index().sort_values("rv")
test[(test["pfx_x"] > 500) & (test["pitch_type"] == "SL")].head(50)

,player_name,pitch_type,rv,pfx_x
830,"Díaz, Edwin O",SL,-0.027370,534
2139,"Muñoz, Andrés",SL,-0.026900,661
2234,"Ohtani, Shohei",SL,-0.022211,984
700,"Darvish, Yu",SL,-0.020545,566
2669,"Scherzer, Max",SL,-0.019808,507
517,"Cease, Dylan",SL,-0.018303,1338
2123,"Murfee, Penn",SL,-0.015831,511
1459,"Jax, Griffin",SL,-0.015327,560
2636,"Sandoval, Patrick",SL,-0.013621,711
191,"Barlow, Scott",SL,-0.013607,534


In [108]:
#train_df =
train_df = full_pitch_df.iloc[0:round(len(full_pitch_df) * 0.64), :]
cv_df = full_pitch_df.iloc[(round(len(full_pitch_df) * 0.64)):round(len(full_pitch_df) * 0.8), :]
test_df = full_pitch_df.iloc[(round(len(full_pitch_df) * 0.8)):, :]
(len(train_df) + len(cv_df) + len(test_df)) == len(full_pitch_df)

True

In [100]:
# Initial test model 
xgb_fit = xgb.XGBRegressor(n_estimators = 300, learning_rate = 0.01, max_depth = 5, tree_method = "hist")
xgb_fit.fit(train_df[pitch_char_list + location_list + context_list], train_df["rv"])

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=300, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [ ]:
SEARCH_PARAMS = {'learning_rate': trial.suggest_uniform("learning_rate", 0.005, 0.3),
                 'max_depth': int(trial.suggest_discrete_uniform("max_depth", 3, 10, 1)),
                 'n_estimators': int(trial.suggest_discrete_uniform("n_estimators", 50, 500, 1)),
                 'lambda_l1':
                 'num_leaves': int(trial.suggest_discrete_uniform("max_depth", 3, 10, 1)),
                 'feature_fraction': trial.suggest_uniform("subsample", 0.5, 0.8),
                 'subsample': trial.suggest_uniform("subsample", 0.5, 0.8)}

FIXED_PARAMS={'objective': 'binary',
             'metric': 'auc',
             'is_unbalance':True,
             'bagging_freq':5,
             'boosting':'dart',
             'early_stopping_rounds':30}


In [ ]:
# Establish hyperparameter range
def objective(trial):
     
    xgb_fit.fit(train_df[pitch_char_list + location_list + context_list], train_df["rv"])
    
    pred = xgb_fit.predict(cv_df[pitch_char_list + location_list + context_list])
    
    return((np.mean((pred - cv_df["rv"])**2))**0.5)
    

In [118]:
# Establish hyperparameter range
def objective(trial):
    
    xgb_fit = xgb.XGBRegressor(n_estimators = int(trial.suggest_float("n_estimators", 50, 1000, step = 1)), 
                               max_depth = int(trial.suggest_float("max_depth", 3, 10, step = 1)),
                               learning_rate = trial.suggest_float("learning_rate", 0.005, 0.3), 
                               subsample = trial.suggest_float("subsample", 0.6, 1),
                               tree_method = "hist")
    
    xgb_fit.fit(train_df[pitch_char_list + location_list + context_list], train_df["rv"])
    
    pred = xgb_fit.predict(cv_df[pitch_char_list + location_list + context_list])
    
    return((np.mean((pred - cv_df["rv"])**2))**0.5)
    

In [119]:
context_list

['balls',
 'strikes',
 'batter_right',
 'p_throws_right',
 'sz_top',
 'sz_bot',
 'tto']

In [120]:
import time
start_time = time.time()
study = optuna.create_study()
 # The optimization finishes after evaluating 1000 times or 3 seconds.
study.optimize(objective, n_trials = 100)
print(time.time() - start_time)


[I 2023-10-23 22:20:28,660] A new study created in memory with name: no-name-127efd14-bdea-4314-a515-09efe2038b05
[I 2023-10-23 22:21:51,565] Trial 0 finished with value: 0.22149429503091442 and parameters: {'n_estimators': 653.0, 'max_depth': 10.0, 'learning_rate': 0.2885033377933189, 'subsample': 0.7115808180522478}. Best is trial 0 with value: 0.22149429503091442.
[I 2023-10-23 22:22:56,487] Trial 1 finished with value: 0.1991002454304466 and parameters: {'n_estimators': 604.0, 'max_depth': 10.0, 'learning_rate': 0.05800037986589545, 'subsample': 0.9781486570259706}. Best is trial 1 with value: 0.1991002454304466.
[I 2023-10-23 22:23:33,999] Trial 2 finished with value: 0.19727358357684313 and parameters: {'n_estimators': 475.0, 'max_depth': 8.0, 'learning_rate': 0.045423481726739885, 'subsample': 0.9250261214736584}. Best is trial 2 with value: 0.19727358357684313.
[I 2023-10-23 22:23:51,892] Trial 3 finished with value: 0.20481540232052137 and parameters: {'n_estimators': 170.0, '

[W 2023-10-24 17:17:01,204] Trial 30 failed with value None.


KeyboardInterrupt: 

In [ ]:
def objective(hyperparam):
    cv_rmse = np.array([0, 0, 0, 0, 0], dtype = "float")
    # Initialize XGBoost
    xgb_fit = xgb.XGBRegressor(n_estimators = int(hyperparam["n_estimators"]), max_depth = int(hyperparam["max_depth"]), 
                               learning_rate = hyperparam["learning_rate"], subsample = hyperparam["subsample"])
    for i in range(5):
        train_iter_df = train_df[train_df["fold"] != (i + 1)]
        cv_df = train_df[train_df["fold"] == (i + 1)]
        
        # Fit on training data, predict on the left out fold, and calculate RMSE
        xgb_fit.fit(train_iter_df[["x1", "x2"]], train_iter_df["y"])
        pred = xgb_fit.predict(cv_df[["x1", "x2"]])
        cv_rmse[i] = (np.mean((pred - cv_df["y"])**2))**0.5
    
    print ("RMSE:", cv_rmse.mean())
    return {"loss": cv_rmse.mean(), "status": STATUS_OK }

In [60]:
# Look at predictions
print(np.mean((xgb_fit.predict(cv_df[pitch_char_list + location_list + context_list]) - cv_df["rv"])**2)**0.5)
print(cv_df["rv"].corr(pd.Series(xgb_fit.predict(cv_df[pitch_char_list + location_list + context_list]))))

0.1989549236771565
0.01181250166912025


In [122]:
train_df["rv"].std()

0.20066377712764996

In [79]:
cv_df["pred"] = xgb_fit.predict(cv_df[pitch_char_list + location_list + context_list])
test = cv_df.groupby(["player_name", "pitch_type"]).aggregate({"pred":"mean", "rv":"count"}).reset_index().sort_values("pred")

test[test["rv"] > 50]

C:\Users\ishaa\AppData\Local\Temp\ipykernel_27704\1015431315.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cv_df["pred"] = xgb_fit.predict(cv_df[pitch_char_list + location_list + context_list])


,player_name,pitch_type,pred,rv
202,"Bleier, Richard",SI,0.013171,70
1176,"Mantiply, Joe",SI,0.013698,66
664,"García, Jarlín",FF,0.015448,86
445,"Darvish, Yu",FC,0.015784,172
1104,"Loup, Aaron",SI,0.015851,65
...,...,...,...,...
1819,"Suter, Brent",CH,0.041027,57
1103,"Loup, Aaron",FC,0.041812,55
47,"Alvarado, José",FC,0.042267,58
619,"Freeland, Kyle",KC,0.042730,82


In [74]:
cv_df.groupby(["player_name", "pitch_type"])["rv"].mean().reset_index().sort_values("rv").head(n = 25)

,player_name,pitch_type,rv
1493,"Plassmeyer, Michael",CH,-0.259186
1997,"Weems, Jordan",CU,-0.259186
1082,"Liberatore, Matthew",SI,-0.243833
553,"Estrada, Jeremiah",CH,-0.233658
614,"Foster, Matt",CU,-0.233658
116,"Bard, Luke",CH,-0.233658
1008,"Knizner, Andrew",SL,-0.233658
1424,"Ortega, Oliver",FF,-0.233658
1322,"Moreta, Dauri",CH,-0.233658
2078,"Yates, Kirby",SL,-0.233658
